In [11]:
import pandas as pd
import sqlite3

## • create a connection to the database using the library sqlite3

In [12]:
con = sqlite3.connect('../data/checking-logs.sqlite.sqlite')


## • using only one query for each of the groups, create two dataframes: test_results and control_results with the columns time and avg_diff and only two rows
◦ time should have the values: after and before
◦ avg_diff contains the average delta among all the users for the time period before each of them made their first visit to the page and afterward
◦ only take into account the users that have observations before and after
## • we still are not using the lab ’project1’

In [13]:
query = """
        SELECT 'before' AS 'Time',
            AVG((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff
        FROM
        (SELECT * FROM
        test
        WHERE labname IS NOT 'project1'
        AND uid IN (SELECT uid FROM test
        WHERE uid IN (SELECT uid FROM test
                       WHERE first_view_ts < first_commit_ts AND labname IS NOT 'project1')
        AND uid IN (SELECT uid FROM test
                       WHERE first_view_ts > first_commit_ts AND labname IS NOT 'project1')
         )) AS q1
         JOIN deadlines ON q1.labname = deadlines.labs
         WHERE first_view_ts > first_commit_ts
         UNION
         SELECT 'after' AS 'Time',
            AVG((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff
        FROM
        (SELECT * FROM
        test
        WHERE labname IS NOT 'project1'
        AND uid IN (SELECT uid FROM test
        WHERE uid IN (SELECT uid FROM test
                       WHERE first_view_ts < first_commit_ts AND labname IS NOT 'project1')
        AND uid IN (SELECT uid FROM test
                       WHERE first_view_ts > first_commit_ts AND labname IS NOT 'project1')
         )) AS q1
         JOIN deadlines ON q1.labname = deadlines.labs
         WHERE first_view_ts < first_commit_ts

        """
test_results = pd.io.sql.read_sql(query, con)
test_results

,Time,avg_diff
0,after,-104.6000
1,before,-60.5625


In [14]:
query = """
        SELECT 'before' AS 'Time',
            AVG((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff
        FROM
        (SELECT * FROM
        control
        WHERE labname IS NOT 'project1'
        AND uid IN (SELECT uid FROM control
        WHERE uid IN (SELECT uid FROM control
                       WHERE first_view_ts < first_commit_ts AND labname IS NOT 'project1')
        AND uid IN (SELECT uid FROM control
                       WHERE first_view_ts > first_commit_ts AND labname IS NOT 'project1')
         )) AS q1
         JOIN deadlines ON q1.labname = deadlines.labs
         WHERE first_view_ts > first_commit_ts
         UNION
         SELECT 'after' AS 'Time',
            AVG((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff
        FROM
        (SELECT * FROM
        control
        WHERE labname IS NOT 'project1'
        AND uid IN (SELECT uid FROM control
        WHERE uid IN (SELECT uid FROM control
                       WHERE first_view_ts < first_commit_ts AND labname IS NOT 'project1')
        AND uid IN (SELECT uid FROM control
                       WHERE first_view_ts > first_commit_ts AND labname IS NOT 'project1')
         )) AS q1
         JOIN deadlines ON q1.labname = deadlines.labs
         WHERE first_view_ts < first_commit_ts

        """

test_results = pd.io.sql.read_sql(query, con)
test_results

,Time,avg_diff
0,after,-117.636364
1,before,-99.464286


## • close the connection

In [15]:
con.close()

## • have the answer: did the hypothesis turn out to be true and the page does affect the students’ behavior?

In first group delta is changing more strongly then in test group